In [1]:
#1) Detect when vehicle is being used  
#2) Match driver's phone and vehicle
#3) Track driving behavior
#4) Track if texting is enabled
#5) Run scoring model, on risky and distracted events
#6) Make data available for Scoring API

In [ ]:
def get_dist_from_trip(trip, max_time_gap=10**10):
    """
    Returns distance traveled during trip (in km)
    
    :param trip: trip dictionary
    :return: (odo_distance, haversine_distance)

In [19]:
import json
import pandas as pd
from dateutil.parser import parse
from datetime import datetime
import math

#Helper Functions
def parse_time(s):
    try:
        ret = parse(s)
    except ValueError:
        ret = datetime.utcfromtimestamp(s)
    return ret

#print(parse_time(data['lastTimestamp'][0]))

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

#distance([-88.3185833333333, 42.228175],[-88.3185833333333, 48.228175])

In [23]:
distance([-88.3185833333333, 42.228175],[-88.3185833333333, 48.228175])

19.567183279711777

In [24]:
haversine(-88.3185833333333, 42.228175, -88.3185833333333, 48.228175)

667.169559867352

In [37]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

center_point = [{'lat': 33.85750, 'lng': -118.28885}] #k1 speed
test_point = [{'lat': 33.85717, 'lng': -118.29022}] #green olive

lat1 = center_point[0]['lat']
lon1 = center_point[0]['lng']
lat2 = test_point[0]['lat']
lon2 = test_point[0]['lng']

radius = 0.01 # in kilometer #0.01 km = 10 meters

a = haversine(lon1, lat1, lon2, lat2)

print('Distance (km) : ', a)
if a <= radius:
    print('Inside the area')
else:
    print('Outside the area')

Distance (km) :  0.13171921744122636
Outside the area


In [28]:
!pip install geopy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
from geopy import distance

center_point = [{'lat': -7.7940023, 'lng': 110.3656535}]
test_point = [{'lat': -7.79457, 'lng': 110.36563}]
radius = 5 # in kilometer

center_point_tuple = tuple(center_point[0].values()) # (-7.7940023, 110.3656535)
test_point_tuple = tuple(test_point[0].values()) # (-7.79457, 110.36563)

dis = distance.distance(center_point_tuple, test_point_tuple).km
print("Distance: {}".format(dis)) # Distance: 0.0628380925748918

if dis <= radius:
    print("{} point is inside the {} km radius from {} coordinate".format(test_point_tuple, radius, center_point_tuple))
else:
    print("{} point is outside the {} km radius from {} coordinate".format(test_point_tuple, radius, center_point_tuple))

ModuleNotFoundError: No module named 'geopy'

In [3]:
#TelemetryByVinController #Header Parameters #x-id-token

#Grab vin, lastTimestamp, location-coordinates, telemetry-speed-value
with open('telemetry_api_response.txt') as f: 
    json_data = json.load(f)
json_data

{'vin': 'VINVINVINVINVIN01',
 'correlationId': 'correlation-id',
 'lastTimestamp': '2018-06-02T22:04:18Z',
 'location': {'type': 'Point', 'coordinates': [-88.3185833333333, 42.228175]},
 'vehicleName': 'CAMRY',
 'telemetry': {'fuelLevel': {'value': 26.46},
  'odometer': {'value': 5176, 'unit': 'mi'},
  'sunRoof': {'value': 2},
  'spareTirePressure': {'value': 127, 'unit': 'psi'},
  'flTirePressure': {'value': 36, 'unit': 'psi'},
  'driverWindow': {'value': 2},
  'rrWindow': {'value': 2},
  'speed': {'value': 0, 'unit': 'km/h'},
  'frTirePressure': {'value': 37, 'unit': 'psi'},
  'rlTirePressure': {'value': 37, 'unit': 'psi'},
  'passengerWindow': {'value': 2},
  'rrTirePressure': {'value': 37, 'unit': 'psi'},
  'rlWindow': {'value': 2}}}

In [25]:
#convert km/h to mph for speed

mph = 0.6214 * kmh

0.6214

In [ ]:
#steering angle not present for cornering.

In [ ]:
#simulate api call 

In [4]:
d = json_data
df = pd.DataFrame.from_dict(d, orient='index').astype(str).sort_index()

df.T #columnn for each timestamp?
data = df.T

In [5]:
df_repeated = pd.concat([data]*3, ignore_index=True)
df_repeated

,correlationId,lastTimestamp,location,telemetry,vehicleName,vin
0,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01
1,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01
2,correlation-id,2018-06-02T22:04:18Z,"{'type': 'Point', 'coordinates': [-88.31858333...","{'fuelLevel': {'value': 26.46}, 'odometer': {'...",CAMRY,VINVINVINVINVIN01


In [6]:
#len(df_repeated.telemetry.values) #convert to array, #length is number of rows. 

In [7]:
odo = []
sp = []

for i in df_repeated.telemetry.values:
    json_acceptable_string = i.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    odo.append(d['odometer'].get("value"))
    sp.append(d['speed'].get("value"))
    
df_repeated['odometer'] = odo
df_repeated['speed'] = sp
#df_repeated

In [8]:
gps_coord = []

for i in df_repeated.location.values:
    json_acceptable_string = i.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    gps_coord.append(d['coordinates']) #no value
    
df_repeated['gps_coord'] = gps_coord

In [10]:
#df_repeated.columns
data = df_repeated[['vin','lastTimestamp', 'speed','gps_coord', 'odometer']]
data['lastTimestamp'] = data['lastTimestamp'].apply(lambda x: parse_time(x))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#speed threshold
#gps_coord - geofencing, match with phone
#odo - distance traveled

In [12]:
##1) Detect when vehicle is being used  
#When Speed > 0 mph

#Tag Driver -- Assign Driver_ID
#Tag Trips -- identify time gaps of > 10 mins , Assign Trip_ID
#Tag Phone -- identify Phone_ID, match gps_coord with phone gps_coord. 

In [13]:
data

,vin,lastTimestamp,speed,gps_coord,odometer
0,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176
1,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176
2,VINVINVINVINVIN01,2018-06-02 22:04:18+00:00,0,"[-88.3185833333333, 42.228175]",5176


In [14]:
#Distracted Driving
#When Speed > 30 mph
#Geofencing logic
#Locked Miles 

In [15]:
#2) Match driver's phone and vehicle

In [16]:
#3) Track driving behavior

In [17]:
#4) Track if texting is enabled

In [18]:
#5) Run scoring model, on risky and distracted events